#Manipulating States & Operators

###Contents
- [Introduction](#intro)
- [State Vectors](#state)
- [Density Matrices](#density)
- [Qubit (Two-level) Systems](#qubit)
- [Expectation Values](#expect)
- [Super Operators & Vectorized Operators](#super)

In [38]:
import numpy as np
from qutip import *

<a id='intro'></a>

##Introduction

In the previous guide section [Basic Operations on Quantum Objects](BasicOperations.ipynb), we saw how to create states and operators, using the functions built into QuTiP. In this portion of the guide, we will look at performing basic operations with states and operators.  For more detailed demonstrations on how to use and manipulate these objects, see the examples on the [tutorials](http://qutip.org/tutorials.html) web page.

<a id='state'></a>

##State Vectors

Here we begin by creating a Fock ground state vector $\left|0\right>$ with in a Hilbert space with 5 number states, from 0 to 4 using the `basis` function:

In [3]:
vac = basis(5, 0)
vac

Quantum object: dims = [[5], [1]], shape = [5, 1], type = ket
Qobj data =
[[ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]

and then create a lowering operator $\left(\hat{a}\right)$ corresponding to 5 number states using the `destroy` function:

In [4]:
a = destroy(5)
a

Quantum object: dims = [[5], [5]], shape = [5, 5], type = oper, isherm = False
Qobj data =
[[ 0.          1.          0.          0.          0.        ]
 [ 0.          0.          1.41421356  0.          0.        ]
 [ 0.          0.          0.          1.73205081  0.        ]
 [ 0.          0.          0.          0.          2.        ]
 [ 0.          0.          0.          0.          0.        ]]

Now lets apply the destruction operator to our vacuum state ``vac``,

In [5]:
 a * vac

Quantum object: dims = [[5], [1]], shape = [5, 1], type = ket
Qobj data =
[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]

We see that, as expected, the vacuum is transformed to the zero vector.  A more interesting example comes from using the adjoint of the lowering operator, the raising operator $\hat{a}^\dagger$:

In [6]:
 a.dag() * vac

Quantum object: dims = [[5], [1]], shape = [5, 1], type = ket
Qobj data =
[[ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]]

The raising operator has in indeed raised the state `vec` from the vacuum to the $\left| 1\right>$ state.  Instead of using the dagger ``Qobj.dag()`` method to raise the state, we could have also used the built in `create` function to make a raising operator:

In [7]:
c = create(5)
c * vac

Quantum object: dims = [[5], [1]], shape = [5, 1], type = ket
Qobj data =
[[ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]]

which does the same thing.  We can raise the vacuum state more than once by successively apply the raising operator:

In [8]:
c * c * vac

Quantum object: dims = [[5], [1]], shape = [5, 1], type = ket
Qobj data =
[[ 0.        ]
 [ 0.        ]
 [ 1.41421356]
 [ 0.        ]
 [ 0.        ]]

or just taking the square of the raising operator $\left(\hat{a}^\dagger\right)^{2}$:

In [9]:
 c ** 2 * vac

Quantum object: dims = [[5], [1]], shape = [5, 1], type = ket
Qobj data =
[[ 0.        ]
 [ 0.        ]
 [ 1.41421356]
 [ 0.        ]
 [ 0.        ]]

Applying the raising operator twice gives the expected $\sqrt{n + 1}$ dependence.  We can use the product of $c * a$ to also apply the number operator to the state vector ``vac``:

In [10]:
c * a * vac

Quantum object: dims = [[5], [1]], shape = [5, 1], type = ket
Qobj data =
[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]

or on the $\left| 1\right>$ state:

In [11]:
c * a * (c * vac)

Quantum object: dims = [[5], [1]], shape = [5, 1], type = ket
Qobj data =
[[ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]]

or the $\left| 2\right>$ state:

In [12]:
c * a * (c**2 * vac)

Quantum object: dims = [[5], [1]], shape = [5, 1], type = ket
Qobj data =
[[ 0.        ]
 [ 0.        ]
 [ 2.82842712]
 [ 0.        ]
 [ 0.        ]]

Notice how in this last example, application of the number operator does not give the expected value $n=2$, but rather $2\sqrt{2}$.  This is because this last state is not normalized to unity as $c\left| n\right> = \sqrt{n+1}\left| n+1\right>$.  Therefore, we should normalize our vector first:

In [13]:
c * a * (c**2 * vac).unit()

Quantum object: dims = [[5], [1]], shape = [5, 1], type = ket
Qobj data =
[[ 0.]
 [ 0.]
 [ 2.]
 [ 0.]
 [ 0.]]

Since we are giving a demonstration of using states and operators, we have done a lot more work than we should have.  For example, we do not need to operate on the vacuum state to generate a higher number Fock state.  Instead we can use the `basis` (or `fock`) function to directly obtain the required state:

In [15]:
ket = basis(5, 2)
ket

Quantum object: dims = [[5], [1]], shape = [5, 1], type = ket
Qobj data =
[[ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 0.]]

Notice how it is automatically normalized.  We can also use the built in `num` operator:

In [17]:
n = num(5)
n

Quantum object: dims = [[5], [5]], shape = [5, 5], type = oper, isherm = True
Qobj data =
[[ 0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  2.  0.  0.]
 [ 0.  0.  0.  3.  0.]
 [ 0.  0.  0.  0.  4.]]

Therefore, instead of ``c * a * (c ** 2 * vac).unit()`` we have:

In [18]:
n * ket

Quantum object: dims = [[5], [1]], shape = [5, 1], type = ket
Qobj data =
[[ 0.]
 [ 0.]
 [ 2.]
 [ 0.]
 [ 0.]]

We can also create superpositions of states:

In [21]:
ket = (basis(5, 0) + basis(5, 1)).unit()
ket

Quantum object: dims = [[5], [1]], shape = [5, 1], type = ket
Qobj data =
[[ 0.70710678]
 [ 0.70710678]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]

where we have used the `Qobj.unit` method to again normalize the state. Operating with the number function again:

In [22]:
n * ket

Quantum object: dims = [[5], [1]], shape = [5, 1], type = ket
Qobj data =
[[ 0.        ]
 [ 0.70710678]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]

We can also create coherent states and squeezed states by applying the `displace` and `squeeze` functions to the vacuum state:

In [23]:
vac = basis(5, 0)
d = displace(5, 1j)
s = squeeze(5, 0.25 + 0.25j)
d * vac

Quantum object: dims = [[5], [1]], shape = [5, 1], type = ket
Qobj data =
[[ 0.60655682+0.j        ]
 [ 0.00000000+0.60628133j]
 [-0.43038740+0.j        ]
 [ 0.00000000-0.24104351j]
 [ 0.14552147+0.j        ]]

In [24]:
d * s * vac

Quantum object: dims = [[5], [1]], shape = [5, 1], type = ket
Qobj data =
[[ 0.65893786+0.08139381j]
 [ 0.10779462+0.51579735j]
 [-0.37567217-0.01326853j]
 [-0.02688063-0.23828775j]
 [ 0.26352814+0.11512178j]]

Of course, displacing the vacuum gives a coherent state, which can also be generated using the built in `coherent` function.

<a id='density'></a>

##Density Matrices

One of the main purpose of QuTiP is to explore the dynamics of **open** quantum systems, where the most general state of a system is not longer a state vector, but rather a density matrix.  Since operations on density matrices operate identically to those of vectors, we will just briefly highlight creating and using these structures.

The simplest density matrix is created by forming the outer-product $\left|\psi\right>\left<\psi\right|$ of a ket vector:

In [26]:
ket = basis(5, 2)
ket * ket.dag()

Quantum object: dims = [[5], [5]], shape = [5, 5], type = oper, isherm = True
Qobj data =
[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]]

A similar task can also be accomplished via the `fock_dm` or `ket2dm` functions:

In [27]:
fock_dm(5, 2)

Quantum object: dims = [[5], [5]], shape = [5, 5], type = oper, isherm = True
Qobj data =
[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]]

In [28]:
ket2dm(ket)

Quantum object: dims = [[5], [5]], shape = [5, 5], type = oper, isherm = True
Qobj data =
[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]]

If we want to create a density matrix with equal classical probability of being found in the $\left|2\right>$ or $\left|4\right>$ number states we can do the following:

In [29]:
0.5 * ket2dm(basis(5, 4)) + 0.5 * ket2dm(basis(5, 2))

Quantum object: dims = [[5], [5]], shape = [5, 5], type = oper, isherm = True
Qobj data =
[[ 0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0. ]
 [ 0.   0.   0.5  0.   0. ]
 [ 0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.5]]

or use ``0.5 * fock_dm(5, 2) + 0.5 * fock_dm(5, 4)``. There are also several other built-in functions for creating predefined density matrices, for example `coherent_dm` and `thermal_dm` which create coherent state and thermal state density matrices, respectively.

In [30]:
coherent_dm(5, 1.25)

Quantum object: dims = [[5], [5]], shape = [5, 5], type = oper, isherm = True
Qobj data =
[[ 0.20980701  0.26141096  0.23509686  0.15572585  0.13390765]
 [ 0.26141096  0.32570738  0.29292109  0.19402805  0.16684347]
 [ 0.23509686  0.29292109  0.26343512  0.17449684  0.1500487 ]
 [ 0.15572585  0.19402805  0.17449684  0.11558499  0.09939079]
 [ 0.13390765  0.16684347  0.1500487   0.09939079  0.0854655 ]]

In [31]:
thermal_dm(5, 1.25)

Quantum object: dims = [[5], [5]], shape = [5, 5], type = oper, isherm = True
Qobj data =
[[ 0.46927974  0.          0.          0.          0.        ]
 [ 0.          0.26071096  0.          0.          0.        ]
 [ 0.          0.          0.14483942  0.          0.        ]
 [ 0.          0.          0.          0.08046635  0.        ]
 [ 0.          0.          0.          0.          0.04470353]]

QuTiP also provides a set of distance metrics for determining how close two density matrix distributions are to each other. Included are the trace distance `tracedist`, fidelity `fidelity`, Hilbert-Schmidt distance `hilbert_dist`, Bures distance `bures_dist`, and Bures angle `bures_angle`.

In [33]:
x = coherent_dm(5, 1.25)
y = coherent_dm(5, 1.25j)  # <-- note the 'j'
z = thermal_dm(5, 0.125)

In [34]:
fidelity(x, x)

1.0000000208397526

In [35]:
tracedist(y, y)

0.0

We also know that for two pure states, the trace distance (T) and the fidelity (F) are related by $T = \sqrt{1 - F^{2}}$.

In [40]:
print(tracedist(y, x), np.sqrt(1 - fidelity(y, x) ** 2))

0.9771565895267291 0.977156570135


For a pure state and a mixed state, $1 - F^{2} \le T$ which can also be verified:

In [41]:
print(1 - fidelity(x, z) ** 2, tracedist(x, z))

0.7782890497791632 0.8559028328862591


<a id='qubit'></a>

##Qubit (Two-level) Systems

Having spent a fair amount of time on basis states that represent harmonic oscillator states, we now move on to qubit, or two-level quantum systems (for example a spin-1/2). To create a state vector corresponding to a qubit system, we use the same `basis`, or `fock`, function with only two levels:

In [42]:
spin = basis(2, 0)

Now at this point one may ask how this state is different than that of a harmonic oscillator in the vacuum state truncated to two energy levels?

In [43]:
vac = basis(2, 0)

At this stage, there is no difference.  This should not be surprising as we called the exact same function twice.  The difference between the two comes from the action of the spin operators `sigmax`, `sigmay`, `sigmaz`, `sigmap`, and `sigmam` on these two-level states.  For example, if ``vac`` corresponds to the vacuum state of a harmonic oscillator, then, as we have already seen, we can use the raising operator to get the $\left|1\right>$ state:

In [44]:
c = create(2)
c * vac

Quantum object: dims = [[2], [1]], shape = [2, 1], type = ket
Qobj data =
[[ 0.]
 [ 1.]]

For a spin system, the operator analogous to the raising operator is the sigma-plus operator `sigmap`.  Operating on the ``spin`` state gives:

In [45]:
sigmap() * spin

Quantum object: dims = [[2], [1]], shape = [2, 1], type = ket
Qobj data =
[[ 0.]
 [ 0.]]

Now we see the difference!  The `sigmap` operator acting on the ``spin`` state returns the zero vector.  Why is this?  To see what happened, let us use the `sigmaz` operator:

In [46]:
sigmaz()

Quantum object: dims = [[2], [2]], shape = [2, 2], type = oper, isherm = True
Qobj data =
[[ 1.  0.]
 [ 0. -1.]]

In [47]:
sigmaz() * spin

Quantum object: dims = [[2], [1]], shape = [2, 1], type = ket
Qobj data =
[[ 1.]
 [ 0.]]

In [48]:
spin2 = basis(2, 1)
sigmaz() * spin2

Quantum object: dims = [[2], [1]], shape = [2, 1], type = ket
Qobj data =
[[ 0.]
 [-1.]]

The answer is now apparent.  Since the QuTiP `sigmaz` function uses the standard z-basis representation of the sigma-z spin operator, the ``spin`` state corresponds to the $\left|\uparrow\right>$ state of a two-level spin system while ``spin2`` gives the $\left|\downarrow\right>$ state.  Therefore, in our previous example ``sigmap() * spin``, we raised the qubit state out of the truncated two-level Hilbert space resulting in the zero state.  

While at first glance this convention might seem somewhat odd, it is in fact quite handy. For one, the spin operators remain in the conventional form. Second, when the spin system is in the $\left|\uparrow\right>$ state:

In [49]:
sigmaz() * spin

Quantum object: dims = [[2], [1]], shape = [2, 1], type = ket
Qobj data =
[[ 1.]
 [ 0.]]

the non-zero component is the zeroth-element of the underlying matrix (remember that python uses c-indexing, and matrices start with the zeroth element).  The $\left|\downarrow\right>$ state therefore has a non-zero entry in the first index position. This corresponds nicely with the quantum information definitions of qubit states, where the excited $\left|\uparrow\right>$ state is label as $\left|0\right>$, and the $\left|\downarrow\right>$ state by $\left|1\right>$.

If one wants to create spin operators for higher spin systems, then the `jmat` function comes in handy. 

<a id='expect'></a>

##Expectation Values

Some of the most important information about quantum systems comes from calculating the expectation value of operators, both Hermitian and non-Hermitian, as the state or density matrix of the system varies in time.  Therefore, in this section we demonstrate the use of the `expect` function.  To begin:


In [50]:
vac = basis(5, 0)
one = basis(5, 1)
c = create(5)
N = num(5)

expect(N, vac)

0.0

In [51]:
expect(N, one)

1.0

In [52]:
coh = coherent_dm(5, 1.0j)
expect(N, coh)

0.9970555745806599

In [53]:
cat = (basis(5, 4) + 1.0j * basis(5, 3)).unit()
expect(c, cat)

0.9999999999999998j

Notice how in this last example, all of the return values are complex numbers.  This is because the `expect` function looks to see whether the operator is Hermitian or not.  If the operator is Hermitian, than the output will always be real.  In the case of non-Hermitian operators, the return values may be complex.  Therefore, the `expect` function will return an array of complex values for non-Hermitian operators when the input is a list/array of states or density matrices.


Of course, the `expect` function works for spin states and operators as well:

In [54]:
up = basis(2, 0)
down = basis(2, 1)

expect(sigmaz(), up)

1.0

In [55]:
expect(sigmaz(), down)

-1.0

as well as the composite objects discussed in the next section ([Tensor Products & Partial Traces](TensorPtrace.ipynb)):

In [56]:
spin1 = basis(2, 0)
spin2 = basis(2, 1)
two_spins = tensor(spin1, spin2)
sz1 = tensor(sigmaz(), qeye(2))
sz2 = tensor(qeye(2), sigmaz())

expect(sz1, two_spins)

1.0

In [57]:
expect(sz2, two_spins)

-1.0

<a id='super'></a>

##Super Operators and Vectorized Operators

In addition to state vectors and density operators, QuTiP allows for
representing maps that act linearly on density operators using the Kraus,
Liouville supermatrix and Choi matrix formalisms. This support is based on the
correspondance between linear operators acting on a Hilbert space, and vectors
in two copies of that Hilbert space,
:$\mathrm{vec} : \mathcal{L}(\mathcal{H}) \to \mathcal{H} \otimes \mathcal{H}$.

This isomorphism is implemented in QuTiP by the `operator_to_vector` and 
`vector_to_operator` functions:

In [59]:
rho = fock_dm(2,0)
vec_rho = operator_to_vector(rho)
vec_rho

Quantum object: dims = [[[2], [2]], [1]], shape = [4, 1], type = operator-ket
Qobj data =
[[ 1.]
 [ 0.]
 [ 0.]
 [ 0.]]

In [60]:
rho2 = vector_to_operator(vec_rho)
(rho - rho2).norm()

0.0

The `Qobj.type` attribute indicates whether a quantum object is
a vector corresponding to an operator (``operator-ket``), or its Hermitian
conjugate (``operator-bra``).

Note that QuTiP uses the *column-stacking* convention for the isomorphism
between $\mathcal{L}(\mathcal{H})$ and $\mathcal{H} \otimes \mathcal{H}$:

In [61]:
A = Qobj(np.arange(4).reshape((2, 2)))
A

Quantum object: dims = [[2], [2]], shape = [2, 2], type = oper, isherm = False
Qobj data =
[[ 0.  1.]
 [ 2.  3.]]

In [62]:
operator_to_vector(A)

Quantum object: dims = [[[2], [2]], [1]], shape = [4, 1], type = operator-ket
Qobj data =
[[ 0.]
 [ 2.]
 [ 1.]
 [ 3.]]

Since $\mathcal{H} \otimes \mathcal{H}$ is a vector space, linear maps
on this space can be represented as matrices, often called *supermatrices*.
Using the `Qobj`, the `spre` and `spost` functions, supermatrices
corresponding to left- and right-multiplication respectively can be quickly
constructed.

In [64]:
X = sigmax()
S = spre(X) * spost(X.dag()) # Represents conjugation by X.

Note that this is done automatically by the `to_super` function when given
``type='oper'`` input.

In [65]:
S2 = to_super(X)
(S - S2).norm()

0.0

Quantum objects representing superoperators are denoted by ``type='super'``:

In [66]:
S

Quantum object: dims = [[[2], [2]], [[2], [2]]], shape = [4, 4], type = super, isherm = True
Qobj data =
[[ 0.  0.  0.  1.]
 [ 0.  0.  1.  0.]
 [ 0.  1.  0.  0.]
 [ 1.  0.  0.  0.]]

Information about superoperators, such as whether they represent completely
positive maps, is exposed through the `iscp`, `Qobj.istp`
and `Qobj.iscptp` attributes:

In [67]:
S.iscp, S.istp, S.iscptp

(True, True, True)

In addition, dynamical generators on this extended space, often called
*Liouvillian superoperators*, can be created using the `liouvillian` function. Each of these takes a Hamilonian along with a list of collapse operators, and returns a ``type="super"`` object that can be exponentiated to find the superoperator for that evolution.

In [68]:
H = 10 * sigmaz()
c1 = destroy(2)
L = liouvillian(H, [c1])
L

Quantum object: dims = [[[2], [2]], [[2], [2]]], shape = [4, 4], type = super, isherm = False
Qobj data =
[[ 0.0 +0.j  0.0 +0.j  0.0 +0.j  1.0 +0.j]
 [ 0.0 +0.j -0.5+20.j  0.0 +0.j  0.0 +0.j]
 [ 0.0 +0.j  0.0 +0.j -0.5-20.j  0.0 +0.j]
 [ 0.0 +0.j  0.0 +0.j  0.0 +0.j -1.0 +0.j]]

In [74]:
S = (12 * L).expm()
S

Quantum object: dims = [[[2], [2]], [[2], [2]]], shape = [4, 4], type = super, isherm = False
Qobj data =
[[  1.00000000e+00+0.j           0.00000000e+00+0.j           0.00000000e+00+0.j
    9.99993856e-01+0.j        ]
 [  0.00000000e+00+0.j           8.07531120e-04+0.00234352j
    0.00000000e+00+0.j           0.00000000e+00+0.j        ]
 [  0.00000000e+00+0.j           0.00000000e+00+0.j
    8.07531120e-04-0.00234352j   0.00000000e+00+0.j        ]
 [  0.00000000e+00+0.j           0.00000000e+00+0.j           0.00000000e+00+0.j
    6.14421235e-06+0.j        ]]

Once a superoperator has been obtained, it can be converted between the
supermatrix, Kraus and Choi formalisms by using the `superop_reps.to_super`,
`superop_reps.to_kraus` and `superop_reps.to_choi` functions. The `Qobj.superrep`
attribute keeps track of what reprsentation is a `Qobj` is currently using.

In [76]:
J = to_choi(S)
J

Quantum object: dims = [[[2], [2]], [[2], [2]]], shape = [4, 4], type = super, isherm = True, superrep = choi
Qobj data =
[[  1.00000000e+00+0.j           0.00000000e+00+0.j           0.00000000e+00+0.j
    8.07531120e-04-0.00234352j]
 [  0.00000000e+00+0.j           0.00000000e+00+0.j           0.00000000e+00+0.j
    0.00000000e+00+0.j        ]
 [  0.00000000e+00+0.j           0.00000000e+00+0.j           9.99993856e-01+0.j
    0.00000000e+00+0.j        ]
 [  8.07531120e-04+0.00234352j   0.00000000e+00+0.j           0.00000000e+00+0.j
    6.14421235e-06+0.j        ]]

In [77]:
K = to_kraus(J)
K

[Quantum object: dims = [[2], [2]], shape = [2, 2], type = oper, isherm = False
 Qobj data =
 [[  1.00000000e+00 +1.34376978e-22j   0.00000000e+00 +0.00000000e+00j]
  [  0.00000000e+00 +0.00000000e+00j   8.07531120e-04 +2.34352424e-03j]],
 Quantum object: dims = [[2], [2]], shape = [2, 2], type = oper, isherm = False
 Qobj data =
 [[ -1.11923759e-13 +6.02807402e-15j   0.00000000e+00 +0.00000000e+00j]
  [  0.00000000e+00 +0.00000000e+00j   1.70093171e-11 +4.18976706e-11j]],
 Quantum object: dims = [[2], [2]], shape = [2, 2], type = oper, isherm = True
 Qobj data =
 [[ 0.  0.]
  [ 0.  0.]],
 Quantum object: dims = [[2], [2]], shape = [2, 2], type = oper, isherm = False
 Qobj data =
 [[ 0.          0.99999693]
  [ 0.          0.        ]]]

In [1]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../styles/guide.css", "r").read()
    return HTML(styles)
css_styling()